In [22]:
""" in mesh object pointer"""

import numpy as np


M_OBJECTS = {}


class M_Object(object):
    def __init__(self, line):  # type: ignore
        self.id = len(M_OBJECTS) + 1
        self.lines = {line.id: np.array(line.coords, dtype=np.float16)}
        # TODO: set this to only run if the object has been updated since last run
        self.dimensions: list = [0, 0, 0, 0]  # min_x, max_x, min_y, max_y
        # ---------------
        self.min_x = self.dimensions[0]
        self.max_x = self.dimensions[1]
        # ---------------
        M_OBJECTS.setdefault(self.id, self)

    def append(self, line):
        self.lines[line.id] = np.array(line.coords, dtype=np.float16)

    def line_min_x(self, id):
        return min([e[0:1] for e in self.lines[id]])

    def line_max_x(self, id):
        return max([e[0:1] for e in self.lines[id]])

    def min_y(self):
        return min(self.lines.keys())

    def max_y(self):
        return max(self.lines.keys())

    def calc_dimensions(self) -> list:
        self_min_x: float = 0
        self_max_x: float = 0

        for line in self.lines.keys():
            min_x: float = min(self.line_min_x(line))
            max_x: float = max(self.line_max_x(line))
            if min_x < self_min_x:
                self_min_x = min_x
            if max_x > self_max_x:
                self_max_x = max_x
        self.dimensions = [self_min_x, self_max_x, self.min_y(), self.max_y()]


""" **in** M_Object , post line sorting"""
# libs

# imports

M_LINES = {}
# =========================


class M_Line(object):
    def __init__(self, id):
        self.id = id
        self.coords = []
        self.min_x = lambda: self.coords[0][0]
        self.max_x = lambda: self.coords[-1][0]

    def append(self, coord):
        self.coords.append(coord)

    def close(self, new_obj: bool = False):
        if len(self.coords) == 0:
            return
        elif new_obj is False:
            keys = max(M_OBJECTS.keys())
            M_OBJECTS[keys].append(self)
        else:
            M_Object(self)

    def q_close(self):
        try:
            min_x = self.min_x()
            max_x = self.max_x()
        except Exception as e:
            # print(f"M_Line.close() error: {e}")
            return
        # ---------
        if len(M_OBJECTS) == 0:
            M_Object(self)
        else:
            for obj in M_OBJECTS:
                print(M_OBJECTS[obj].min_x, M_OBJECTS[obj].max_x)
                if (
                    min_x >= M_OBJECTS[obj].min_x - 0.2
                    and max_x <= M_OBJECTS[obj].max_x + 0.2
                ):
                    M_OBJECTS[obj].append(self)
                    break
                else:
                    M_Object(self)
                    break


first_m_line = M_Line(1)
first_m_line.append([1, 2, 3])
second_m_line = M_Line(2)
second_m_line.append([5, 6, 7])
first_m = M_Object(first_m_line)
first_m.calc_dimensions()
print(first_m.dimensions)
first_m.append(second_m_line)
first_m.calc_dimensions()
print(first_m.dimensions)


[0, 1.0, 1, 1]
[0, 5.0, 1, 2]
None


In [5]:
import numpy as np

diff = np.diff((0.6, -0.2), 1)
print(diff)

[-0.8]
